# Learning LSTM Autoencoder and LSTM Network on a simple Multivariate Timeseries Toy example

In [40]:
# lstm autoencoder to recreate a timeseries
import pandas as pd
import numpy as np
import ta
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.layers import Input
from keras.models import Model
from sklearn.preprocessing import MinMaxScaler

In [41]:
'''
A UDF to convert input data into 3-D
array as required for LSTM network.
'''

def temporalize(X, y, lookback):
    output_X = []
    output_y = []
    for i in range(len(X)-lookback-1):
        t = []
        for j in range(1,lookback+1):
            # Gather past records upto the lookback period
            t.append(X[[(i+j+1)], :])
        output_X.append(t)
        output_y.append(y[i+lookback+1])
    return output_X, output_y

In [42]:
csv_path = '../data/test_1h.csv'
df = pd.read_csv(csv_path, sep=',')
df = df.drop(columns=['quote_asset_volume', 'number_of_trades',
                      'buy_base_asset_volume', 'buy_quote_asset_volume', 'open_time', 'close_time', 'ignore'], axis=1)
df = ta.utils.dropna(df)
train_data = df.copy()

data = ta.add_all_ta_features(
    train_data, open='open', high='high', low='low',
    close='close', volume='volume', fillna=True
)

raw_price = data.close.astype('float64').values
timeseries = data.drop(columns=['open', 'high', 'low', 'close', 'volume'], axis=1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.fit(timeseries)
timeseries = scaler.transform(timeseries)
timeseries.shape

c:\users\thinhpc\anaconda3\envs\autoencoder\lib\site-packages\ta\trend.py:608: RuntimeWarning: invalid value encountered in double_scalars
  dip[i] = 100 * (self._dip[i]/self._trs[i])
c:\users\thinhpc\anaconda3\envs\autoencoder\lib\site-packages\ta\trend.py:612: RuntimeWarning: invalid value encountered in double_scalars
  din[i] = 100 * (self._din[i]/self._trs[i])


(1641, 72)

In [43]:
# # define input timeseries
# timeseries = np.array([[0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
#                        [0.1**3, 0.2**3, 0.3**3, 0.4**3, 0.5**3, 0.6**3, 0.7**3, 0.8**3, 0.9**3]])
# timeseries = timeseries.transpose()
timesteps = timeseries.shape[0]
n_features = timeseries.shape[1]
timeseries.shape

(1641, 72)

In [44]:
timesteps = 10
X, y = temporalize(X=timeseries, y=np.zeros(len(timeseries)), lookback = timesteps)

n_features = 72
X = np.array(X)
X = X.reshape(X.shape[0], timesteps, n_features)

X.shape

(1630, 10, 72)

## LSTM Autoencoder

In [48]:
# define model
inputs = Input(shape=(timesteps, n_features))

encoded = LSTM(1024, activation='relu', return_sequences=True)(inputs)
encoded = LSTM(128, activation='relu', return_sequences=False)(encoded)

decoded = RepeatVector(timesteps)(encoded)
decoded = LSTM(128, activation='relu', return_sequences=True)(decoded)
decoded = LSTM(1024, activation='relu', return_sequences=True)(decoded)
decoded = TimeDistributed(Dense(n_features))(decoded)

model = Model(inputs, decoded)
model.compile(optimizer='adam', loss='mse')
model.summary()

encoder = Model(inputs, encoded)
encoder.summary()

Model: "model_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 10, 72)            0         
_________________________________________________________________
lstm_25 (LSTM)               (None, 10, 1024)          4493312   
_________________________________________________________________
lstm_26 (LSTM)               (None, 128)               590336    
_________________________________________________________________
repeat_vector_7 (RepeatVecto (None, 10, 128)           0         
_________________________________________________________________
lstm_27 (LSTM)               (None, 10, 128)           131584    
_________________________________________________________________
lstm_28 (LSTM)               (None, 10, 1024)          4722688   
_________________________________________________________________
time_distributed_6 (TimeDist (None, 10, 72)            7380

In [49]:
# fit model
model.fit(X, X, epochs=2000, batch_size=128, verbose=1)
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
# print('---Predicted---')
# print(np.round(yhat,3))
# print('---Actual---')
# print(np.round(X, 3))

Epoch 1/2000
1630/1630 [==============================] - 4s 2ms/step - loss: 2.4473
Epoch 2/2000
1630/1630 [==============================] - 2s 937us/step - loss: 0.2576
Epoch 3/2000
1630/1630 [==============================] - 2s 938us/step - loss: 0.1423
Epoch 4/2000
1630/1630 [==============================] - 2s 936us/step - loss: 0.0691
Epoch 5/2000
1630/1630 [==============================] - 2s 948us/step - loss: 0.0303
Epoch 6/2000
1630/1630 [==============================] - 2s 944us/step - loss: 0.0238
Epoch 7/2000
1630/1630 [==============================] - 2s 945us/step - loss: 0.0211
Epoch 8/2000
1630/1630 [==============================] - 2s 940us/step - loss: 0.0169
Epoch 9/2000
1630/1630 [==============================] - 2s 941us/step - loss: 0.0144
Epoch 10/2000
1630/1630 [==============================] - 2s 937us/step - loss: 0.0132
Epoch 11/2000
1630/1630 [==============================] - 2s 948us/step - loss: 0.0125
Epoch 12/2000
1630/1630 [==================

1630/1630 [==============================] - 2s 942us/step - loss: 0.0041
Epoch 95/2000
1630/1630 [==============================] - 2s 979us/step - loss: 0.0040
Epoch 96/2000
1630/1630 [==============================] - 2s 1ms/step - loss: 0.0040
Epoch 97/2000
1630/1630 [==============================] - 2s 989us/step - loss: 0.0041
Epoch 98/2000
1630/1630 [==============================] - 2s 952us/step - loss: 0.0039
Epoch 99/2000
1630/1630 [==============================] - 2s 981us/step - loss: 0.0039
Epoch 100/2000
1630/1630 [==============================] - 2s 957us/step - loss: 0.0039
Epoch 101/2000
1630/1630 [==============================] - 2s 936us/step - loss: 0.0038
Epoch 102/2000
1630/1630 [==============================] - 2s 934us/step - loss: 0.0038
Epoch 103/2000
1630/1630 [==============================] - 2s 937us/step - loss: 0.0038
Epoch 104/2000
1630/1630 [==============================] - 2s 932us/step - loss: 0.0039
Epoch 105/2000
1630/1630 [=================

1630/1630 [==============================] - 2s 937us/step - loss: 0.0024
Epoch 187/2000
1630/1630 [==============================] - 2s 935us/step - loss: 0.0023
Epoch 188/2000
1630/1630 [==============================] - 2s 933us/step - loss: 0.0022
Epoch 189/2000
1630/1630 [==============================] - 2s 940us/step - loss: 0.0022
Epoch 190/2000
1630/1630 [==============================] - 2s 938us/step - loss: 0.0022
Epoch 191/2000
1630/1630 [==============================] - 2s 935us/step - loss: 0.0024
Epoch 192/2000
1630/1630 [==============================] - 2s 937us/step - loss: 0.0023
Epoch 193/2000
1630/1630 [==============================] - 2s 938us/step - loss: 0.0023
Epoch 194/2000
1630/1630 [==============================] - 2s 953us/step - loss: 0.0022
Epoch 195/2000
1630/1630 [==============================] - 2s 934us/step - loss: 0.0022
Epoch 196/2000
1630/1630 [==============================] - 2s 938us/step - loss: 0.0021
Epoch 197/2000
1630/1630 [==========

1630/1630 [==============================] - 2s 949us/step - loss: 0.0012
Epoch 279/2000
1630/1630 [==============================] - 2s 944us/step - loss: 0.0012
Epoch 280/2000
1630/1630 [==============================] - 2s 945us/step - loss: 0.0012
Epoch 281/2000
1630/1630 [==============================] - 2s 941us/step - loss: 0.0013
Epoch 282/2000
1630/1630 [==============================] - 2s 944us/step - loss: 0.0014
Epoch 283/2000
1630/1630 [==============================] - 2s 944us/step - loss: 0.0013
Epoch 284/2000
1630/1630 [==============================] - 2s 930us/step - loss: 0.0012
Epoch 285/2000
1630/1630 [==============================] - 2s 935us/step - loss: 0.0013
Epoch 286/2000
1630/1630 [==============================] - 2s 938us/step - loss: 0.0013
Epoch 287/2000
1630/1630 [==============================] - 2s 943us/step - loss: 0.0014
Epoch 288/2000
1630/1630 [==============================] - 2s 929us/step - loss: 0.0013
Epoch 289/2000
1630/1630 [==========

1630/1630 [==============================] - 2s 937us/step - loss: 7.9989e-04
Epoch 369/2000
1630/1630 [==============================] - 2s 934us/step - loss: 7.5373e-04
Epoch 370/2000
1630/1630 [==============================] - 2s 937us/step - loss: 7.2491e-04
Epoch 371/2000
1630/1630 [==============================] - 2s 933us/step - loss: 7.1971e-04
Epoch 372/2000
1630/1630 [==============================] - 2s 959us/step - loss: 7.2974e-04
Epoch 373/2000
1630/1630 [==============================] - 2s 939us/step - loss: 7.5507e-04
Epoch 374/2000
1630/1630 [==============================] - 2s 933us/step - loss: 7.5678e-04
Epoch 375/2000
1630/1630 [==============================] - 2s 935us/step - loss: 8.6059e-04
Epoch 376/2000
1630/1630 [==============================] - 2s 934us/step - loss: 0.0011
Epoch 377/2000
1630/1630 [==============================] - 2s 932us/step - loss: 0.0011
Epoch 378/2000
1630/1630 [==============================] - 2s 933us/step - loss: 0.0010
Epoc

...

1630/1630 [==============================] - 2s 932us/step - loss: 5.6577e-05
Epoch 1594/2000
1630/1630 [==============================] - 2s 934us/step - loss: 5.5173e-05
Epoch 1595/2000
1630/1630 [==============================] - 2s 930us/step - loss: 5.3728e-05
Epoch 1596/2000
1630/1630 [==============================] - 2s 934us/step - loss: 5.2574e-05
Epoch 1597/2000
1630/1630 [==============================] - 2s 937us/step - loss: 5.1722e-05
Epoch 1598/2000
1630/1630 [==============================] - 2s 934us/step - loss: 5.0999e-05
Epoch 1599/2000
1630/1630 [==============================] - 2s 935us/step - loss: 5.0077e-05
Epoch 1600/2000
1630/1630 [==============================] - 2s 931us/step - loss: 4.9418e-05
Epoch 1601/2000
1630/1630 [==============================] - 2s 933us/step - loss: 4.8690e-05
Epoch 1602/2000
1630/1630 [==============================] - 2s 929us/step - loss: 4.7372e-05
Epoch 1603/2000
1630/1630 [==============================] - 2s 930us/step -

1630/1630 [==============================] - 2s 938us/step - loss: 1.0917e-04
Epoch 1681/2000
1630/1630 [==============================] - 2s 943us/step - loss: 1.3168e-04
Epoch 1682/2000
1630/1630 [==============================] - 2s 975us/step - loss: 9.6160e-05
Epoch 1683/2000
1630/1630 [==============================] - 2s 942us/step - loss: 7.2432e-05
Epoch 1684/2000
1630/1630 [==============================] - 2s 945us/step - loss: 6.2251e-05
Epoch 1685/2000
1630/1630 [==============================] - 2s 946us/step - loss: 5.3570e-05
Epoch 1686/2000
1630/1630 [==============================] - 2s 948us/step - loss: 5.3677e-05
Epoch 1687/2000
1630/1630 [==============================] - 2s 950us/step - loss: 4.6168e-05
Epoch 1688/2000
1630/1630 [==============================] - 2s 964us/step - loss: 4.2277e-05
Epoch 1689/2000
1630/1630 [==============================] - 2s 935us/step - loss: 3.9762e-05
Epoch 1690/2000
1630/1630 [==============================] - 2s 979us/step -

1630/1630 [==============================] - 2s 935us/step - loss: 3.2857e-05
Epoch 1768/2000
1630/1630 [==============================] - 2s 933us/step - loss: 3.2475e-05
Epoch 1769/2000
1630/1630 [==============================] - 2s 939us/step - loss: 3.2225e-05
Epoch 1770/2000
1630/1630 [==============================] - 2s 936us/step - loss: 3.2179e-05
Epoch 1771/2000
1630/1630 [==============================] - 2s 967us/step - loss: 3.2609e-05
Epoch 1772/2000
1630/1630 [==============================] - 2s 947us/step - loss: 3.2095e-05
Epoch 1773/2000
1630/1630 [==============================] - 2s 969us/step - loss: 3.2016e-05
Epoch 1774/2000
1630/1630 [==============================] - 2s 961us/step - loss: 3.2550e-05
Epoch 1775/2000
1630/1630 [==============================] - 2s 984us/step - loss: 3.2247e-05
Epoch 1776/2000
1630/1630 [==============================] - 2s 968us/step - loss: 3.3018e-05
Epoch 1777/2000
1630/1630 [==============================] - 2s 940us/step -

1630/1630 [==============================] - 2s 937us/step - loss: 3.3340e-05
Epoch 1855/2000
1630/1630 [==============================] - 2s 937us/step - loss: 3.2733e-05
Epoch 1856/2000
1630/1630 [==============================] - 2s 932us/step - loss: 3.2636e-05
Epoch 1857/2000
1630/1630 [==============================] - 2s 932us/step - loss: 3.1549e-05
Epoch 1858/2000
1630/1630 [==============================] - 2s 940us/step - loss: 3.1944e-05
Epoch 1859/2000
1630/1630 [==============================] - 2s 935us/step - loss: 3.1409e-05
Epoch 1860/2000
1630/1630 [==============================] - 2s 936us/step - loss: 3.0318e-05
Epoch 1861/2000
1630/1630 [==============================] - 2s 938us/step - loss: 2.9267e-05
Epoch 1862/2000
1630/1630 [==============================] - 2s 933us/step - loss: 2.8883e-05
Epoch 1863/2000
1630/1630 [==============================] - 2s 935us/step - loss: 2.9382e-05
Epoch 1864/2000
1630/1630 [==============================] - 2s 933us/step -

1630/1630 [==============================] - 2s 937us/step - loss: 2.9801e-05
Epoch 1942/2000
1630/1630 [==============================] - 2s 942us/step - loss: 3.0009e-05
Epoch 1943/2000
1630/1630 [==============================] - 2s 941us/step - loss: 2.9581e-05
Epoch 1944/2000
1630/1630 [==============================] - 2s 932us/step - loss: 2.9356e-05
Epoch 1945/2000
1630/1630 [==============================] - 2s 933us/step - loss: 2.8971e-05
Epoch 1946/2000
1630/1630 [==============================] - 2s 932us/step - loss: 2.8795e-05
Epoch 1947/2000
1630/1630 [==============================] - 2s 934us/step - loss: 2.8372e-05
Epoch 1948/2000
1630/1630 [==============================] - 2s 933us/step - loss: 2.8046e-05
Epoch 1949/2000
1630/1630 [==============================] - 2s 940us/step - loss: 2.7956e-05
Epoch 1950/2000
1630/1630 [==============================] - 2s 931us/step - loss: 2.7560e-05
Epoch 1951/2000
1630/1630 [==============================] - 2s 944us/step -

In [50]:
np.round(yhat[0][0],2)

array([ 0.14,  0.76,  0.8 ,  0.77,  0.99,  0.77,  0.94,  0.47,  0.35,
        0.82, -0.  ,  0.82,  0.8 ,  0.84,  0.02,  0.63, -0.  ,  0.  ,
        0.82,  0.81,  0.83,  0.03,  0.61,  0.  , -0.  ,  0.01,  0.81,
        0.74,  0.73,  0.59,  0.82,  0.82,  0.81,  0.82, -0.  , -0.  ,
       -0.01,  0.09,  0.03,  0.54,  0.07,  0.07,  0.66, -0.  ,  1.  ,
        0.99,  0.51,  0.82,  0.82,  0.81,  0.83,  0.57,  0.58,  0.08,
        0.01,  0.53, -0.02,  0.89,  0.  ,  1.02,  1.  ,  1.  ,  0.7 ,
        0.83,  0.87,  0.83,  0.73,  0.83,  0.47,  0.51,  0.56,  0.83],
      dtype=float32)

In [51]:
np.round(X[0][0], 2)

array([0.13, 0.76, 0.79, 0.77, 1.  , 0.77, 0.95, 0.47, 0.35, 0.83, 0.  ,
       0.83, 0.8 , 0.85, 0.02, 0.63, 0.  , 0.  , 0.82, 0.81, 0.84, 0.02,
       0.6 , 0.  , 0.  , 0.01, 0.82, 0.74, 0.73, 0.59, 0.83, 0.83, 0.82,
       0.82, 0.  , 0.  , 0.  , 0.09, 0.02, 0.55, 0.08, 0.07, 0.67, 0.  ,
       1.  , 1.  , 0.51, 0.83, 0.83, 0.82, 0.84, 0.57, 0.59, 0.08, 0.  ,
       0.54, 0.  , 0.89, 0.  , 1.  , 1.  , 1.  , 0.69, 0.82, 0.86, 0.82,
       0.73, 0.83, 0.47, 0.52, 0.56, 0.84])

In [52]:
encoder.save_weights("models/autoencoder_weight.h5")
encoder.save("models/model.h5")

In [27]:
from keras.models import load_model
saved_model = load_model("models/model.h5")

In [53]:
yhat = saved_model.predict(X, verbose=0)
np.round(yhat[0][0],2)

array([ 0.13,  0.76,  0.79,  0.78,  1.  ,  0.78,  0.95,  0.46,  0.35,
        0.83, -0.  ,  0.82,  0.8 ,  0.84,  0.02,  0.64,  0.  ,  0.  ,
        0.82,  0.82,  0.84,  0.02,  0.6 ,  0.  , -0.  ,  0.01,  0.82,
        0.75,  0.73,  0.59,  0.83,  0.83,  0.82,  0.82, -0.  ,  0.  ,
       -0.  ,  0.1 ,  0.03,  0.56,  0.08,  0.07,  0.67,  0.02,  1.  ,
        1.  ,  0.52,  0.83,  0.82,  0.82,  0.84,  0.56,  0.59,  0.09,
        0.01,  0.54,  0.02,  0.9 ,  0.  ,  1.  ,  1.01,  1.  ,  0.69,
        0.81,  0.86,  0.81,  0.73,  0.83,  0.47,  0.51,  0.56,  0.84],
      dtype=float32)

In [75]:
np.round(X_train[0],2)

array([0.  , 1.72, 0.  , 3.65, 0.  , 1.51, 0.  , 0.  , 0.03, 0.19, 1.1 ,
       0.05, 0.  , 0.64, 0.  , 0.  , 0.01, 0.  , 0.06, 0.03, 0.23, 0.95,
       0.  , 1.  , 0.  , 3.31, 0.1 , 4.29, 0.  , 0.  , 0.21, 0.  , 0.  ,
       0.  , 0.  , 0.  , 0.  , 0.48, 0.47, 0.  , 1.14, 0.01, 0.  , 0.08,
       0.32, 0.18, 0.  , 0.02, 0.  , 0.  , 0.27, 0.  , 0.72, 0.85, 2.  ,
       0.  , 0.01, 0.  , 0.08, 0.  , 0.  , 0.  , 0.  , 2.37, 0.  , 0.  ,
       7.63, 0.  , 0.01, 0.43, 0.27, 0.  , 8.05, 0.25, 0.89, 1.28, 2.43,
       0.  , 0.  , 7.42, 0.  , 0.02, 0.36, 1.61, 0.  , 1.16, 0.63, 0.  ,
       0.  , 0.12, 0.63, 0.  , 0.67, 1.73, 0.2 , 0.01, 0.  , 0.  , 0.25,
       0.  , 0.01, 0.  , 0.  , 1.43, 0.  , 0.  , 0.  , 0.94, 0.  , 0.  ,
       0.  , 1.09, 0.  , 0.  , 0.  , 3.77, 2.15, 0.44, 0.  , 0.  , 2.64,
       1.64, 0.  , 0.22, 5.92, 0.  , 0.  , 3.85], dtype=float32)

In [77]:
from sklearn.cluster import KMeans
X_train = encoder.predict(X)
clustering = KMeans(n_clusters=3).fit(X_train)
clustering.labels_

array([2, 2, 2, ..., 0, 0, 0])

In [79]:
from pprint import pprint
print(clustering.)
pprint(clustering.labels_)

[[ 5.67487441e-04  2.95275748e-01  5.85599616e-03  7.82028794e-01
   0.00000000e+00  1.32898498e+00  0.00000000e+00  2.45000333e-01
   4.26336657e-03  3.08712959e-01  1.34640813e+00  4.40319598e-01
   0.00000000e+00  4.93087769e-01  0.00000000e+00  0.00000000e+00
   1.91093826e+00  3.03810418e-01  1.28267124e-01  8.77334356e-01
   8.16483498e-01  3.97249848e-01  0.00000000e+00  4.28500473e-01
   0.00000000e+00  9.85926867e-01  1.69959128e-01  1.56231850e-01
   7.01879762e-05  2.18358310e-03  1.69457480e-01  0.00000000e+00
   0.00000000e+00  2.08438826e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  4.46489006e-01  8.02894354e-01  0.00000000e+00
   2.31633735e+00  2.30721876e-01  0.00000000e+00  2.02431247e-01
   1.69939175e-01  1.28257513e-01  0.00000000e+00  4.55501825e-02
   0.00000000e+00  3.96220184e-06  1.46935239e-01  6.25678012e-03
   2.38006637e-01  5.38057268e-01  1.97720313e+00  0.00000000e+00
   6.35671020e-01  0.00000000e+00  2.96890080e-01  8.94887000e-03
   0.00000

In [76]:
for x in clustering.labels_:
    print(x)

0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0


## Regular LSTM Network

In [7]:
# define model
model = Sequential()
model.add(LSTM(128, activation='relu', input_shape=(timesteps,n_features), return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(64, activation='relu', return_sequences=True))
model.add(LSTM(128, activation='relu', return_sequences=True))
model.add(TimeDistributed(Dense(n_features)))
model.compile(optimizer='adam', loss='mse')
model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_5 (LSTM)                (None, 3, 128)            67072     
_________________________________________________________________
lstm_6 (LSTM)                (None, 3, 64)             49408     
_________________________________________________________________
lstm_7 (LSTM)                (None, 3, 64)             33024     
_________________________________________________________________
lstm_8 (LSTM)                (None, 3, 128)            98816     
_________________________________________________________________
time_distributed_2 (TimeDist (None, 3, 2)              258       
Total params: 248,578
Trainable params: 248,578
Non-trainable params: 0
_________________________________________________________________


In [8]:
# fit model
model.fit(X, X, epochs=300, batch_size=5, verbose=0)
# demonstrate reconstruction
yhat = model.predict(X, verbose=0)
print('---Predicted---')
print(np.round(yhat,3))
print('---Actual---')
print(np.round(X, 3))

---Predicted---
[[[0.356 0.063]
  [0.405 0.038]
  [0.506 0.109]]

 [[0.411 0.097]
  [0.5   0.124]
  [0.599 0.222]]

 [[0.483 0.145]
  [0.594 0.224]
  [0.695 0.355]]

 [[0.58  0.211]
  [0.699 0.352]
  [0.795 0.518]]

 [[0.71  0.3  ]
  [0.804 0.511]
  [0.906 0.721]]]
---Actual---
[[[0.3   0.027]
  [0.4   0.064]
  [0.5   0.125]]

 [[0.4   0.064]
  [0.5   0.125]
  [0.6   0.216]]

 [[0.5   0.125]
  [0.6   0.216]
  [0.7   0.343]]

 [[0.6   0.216]
  [0.7   0.343]
  [0.8   0.512]]

 [[0.7   0.343]
  [0.8   0.512]
  [0.9   0.729]]]
